# Complex Polyhedral Sites Setup and Analysis

[TODO]

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.core import Structure, Lattice
import numpy as np
from collections import Counter
from site_analysis import TrajectoryBuilder

In [2]:
# Create a reference structure with the argyrodite topology
# with P occupying the t0 tetrahedra
# dummy atoms occupying the t1, t2, t3, t4, and t5 tetrahedra
# and S occupying all the anion sites

lattice = Lattice.cubic(a=10.155)

coords = np.array(
    [[0.5,     0.5,     0.5],     # P (t0)
     [0.9,     0.9,     0.6],     # t1
     [0.23,    0.92,    0.09],    # t2
     [0.25,    0.25,    0.25],    # t3
     [0.15,    0.15,    0.15],    # t4
     [0.0,     0.183,   0.183],   # t5
     [0.0,     0.0,     0.0],     # S
     [0.75,    0.25,    0.25],    # S
     [0.11824, 0.11824, 0.38176]] # S
) 

reference_structure = Structure.from_spacegroup(sg="F-43m",
                          lattice=lattice,
                          species=['P', 'Li', 'Mg', 'Na', 'Be', 'K', 'S', 'S', 'S'],
                          coords = coords,) * [2,2,2]

In [3]:
# The argyrodite structure contains 6 symmetry inequivalent tetrahedral sites
# Type 0 sites are occupied by P.
# The other tetrahedral site types are possible sites for occupation by Li
# This function uses the site_analysis TrajectoryBuilder to build a Trajectory object
# Note the repeated calls to `with_polyhedral_sites` to define each site type.

def build_trajectory(structure):
    builder = TrajectoryBuilder()
    builder.with_reference_structure(reference_structure) # Our reference structure
    builder.with_structure(structure) # One of the real structures from the MD trajectory
    builder.with_mobile_species('Li')
    builder.with_polyhedral_sites( 
                centre_species = 'Li', # The type 1 sites have Li occupying them in our reference structure
                vertex_species = 'S',
                cutoff = 3.0,
                n_vertices = 4,
                label = 'type 1')
    builder.with_polyhedral_sites(
                centre_species = 'Mg',  # The type 2 sites have Mg occupying them in our reference structure
                vertex_species = 'S',
                cutoff = 3.0,
                n_vertices = 4,
                label = 'type 2')
    builder.with_polyhedral_sites(
                centre_species = 'Na', # The type 3 sites have Na occupying them in our reference structure
                vertex_species = 'S',
                cutoff = 3.0,
                n_vertices = 4,
                label = 'type 3')
    builder.with_polyhedral_sites(
                centre_species = 'Be',  # The type 4 sites have Be occupying them in our reference structure
                vertex_species = 'S',
                cutoff = 3.0,
                n_vertices = 4,
                label = 'type 4')
    builder.with_polyhedral_sites(
                centre_species = 'K',  # The type 5 sites have K occupying them in our reference structure
                vertex_species = 'S',
                cutoff = 3.0,
                n_vertices = 4,
                label = 'type 5')
    # We align the reference structure with our MD structure using the P positions
    builder.with_structure_alignment(align_species='P') 
    builder.with_site_mapping(mapping_species=['S', 'Cl']) 
    # The polyhedral sites are internally defined by their vertex atoms
    # In the reference structure, the tetrahedral vertices are all S atoms, 
    # but in the real materials these tetrahedral vertices can be either S or Cl, 
    # so we perform vertex mapping using both species
    trajectory = builder.build()
    return trajectory

In [4]:
def print_site_occupations(trajectory):
    """
    Print the percentage occupation for each site type in the trajectory.
    
    Args:
        trajectory: A Trajectory object containing atoms and sites information.
    """
    site_types = ['type 5',
                  'type 4',
                  'type 3',
                  'type 2',
                  'type 1']
    site_labels = []
    for atom in trajectory.atoms:
        for i in atom.trajectory:
            site_labels.append(trajectory.sites[i].label)
    
    # Initialize counter with zeros for all site types
    c = Counter()
    for t in site_types:
        c[t] = 0
    
    # Update counter with observed site labels
    c.update(site_labels)
    
    # Calculate and print percentages
    total_sites = sum(c.values())
    for t in site_types:
        print(f'{t}: {(c[t] / total_sites * 100):.2f} %')
        
  

In [5]:
# Li6PS5Cl fully anion ordered (0% anion site exchange)
md_structures = Xdatcar('data/Li6PS5Cl_0p_XDATCAR.gzXDATCAR.gz').structures
trajectory = build_trajectory(md_structures[0])
trajectory.trajectory_from_structures(md_structures, progress=True)
print_site_occupations(trajectory)

100%|█████████████████████████████████████████████████████████████████████| 140/140 [01:28<00:00,  1.58 steps/s]

type 5: 80.20 %
type 4: 0.02 %
type 3: 0.00 %
type 2: 19.78 %
type 1: 0.01 %


In [6]:
# Li6PS5Cl 50% anion site exchange
md_structures = Xdatcar('data/Li6PS5Cl_50p_XDATCAR.gzz').structures
trajectory = build_trajectory(md_structures[0])
trajectory.trajectory_from_structures(md_structures, progress=True)
print_site_occupations(trajectory)

100%|█████████████████████████████████████████████████████████████████████| 140/140 [01:18<00:00,  1.77 steps/s]

type 5: 65.92 %
type 4: 2.63 %
type 3: 0.00 %
type 2: 31.43 %
type 1: 0.02 %


In [7]:
# Li6PS5Cl 100% anion site exchange
md_structures = Xdatcar('data/Li6PS5Cl_100p_XDATCAR.gz').structures
trajectory = build_trajectory(md_structures[0])
trajectory.trajectory_from_structures(md_structures, progress=True)
print_site_occupations(trajectory)

100%|█████████████████████████████████████████████████████████████████████| 140/140 [01:23<00:00,  1.68 steps/s]

type 5: 53.39 %
type 4: 7.33 %
type 3: 0.00 %
type 2: 39.28 %
type 1: 0.00 %
